In [11]:
"""
Adapted from keras example cifar10_cnn.py
Train ResNet-18 on the CIFAR10 small images dataset.

GPU run command with Theano backend (with TensorFlow, the GPU is automatically used):
    THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python cifar10.py
"""

from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
import numpy as np

import resnet

import sys
sys.path.insert(0, '/home/ubuntu/fish/scripts')
# Read in Libraries
from __future__ import division, print_function
from logbook import Logger, StreamHandler
import sys
StreamHandler(sys.stdout).push_application()
log = Logger('Logbook')
import shutil, csv, time
timestr = time.strftime("%Y%m%d")
import ujson as json
import utils; reload(utils)
from utils import *
import gc
# from __future__ import division, print_function
from theano.sandbox import cuda
from vgg16bn import Vgg16BN
from sklearn import metrics
%matplotlib inline

def accuracyfunc(y_act, y_pred):
    return metrics.accuracy_score(np.argmax(y_act, axis=1), np.argmax(y_pred, axis=1))
    
def refresh_directory_structure(name, sub_dirs):
    gdir = os.path.join(path, name)
    if os.path.exists(gdir):
        shutil.rmtree(gdir)
    os.makedirs(gdir)
    for sub_dir in sub_dirs:
        os.makedirs(os.path.join(gdir, sub_dir))

In [12]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
# lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=.5, cooldown=0, patience=5, min_lr=0.5e-7)
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10)
csv_logger = CSVLogger('resnet18_cropped_fish.csv')

batch_size = 32
nb_classes = 8 # 10
nb_epoch = 200
data_augmentation = False
path = "../data/fish/relabel/"

# input image dimensions
img_rows, img_cols = 250, 250
# The CIFAR10 images are RGB.
img_channels = 3

In [13]:
trn = get_data(path+'train', (img_rows, img_cols))
val = get_data(path+'valid', (img_rows, img_cols))

Found 3070 images belonging to 8 classes.
Found 689 images belonging to 8 classes.


In [14]:
trn.shape

(3070, 3, 250, 250)

In [15]:
# get labels
(val_classes, trn_classes, val_labels, trn_labels,
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 3070 images belonging to 8 classes.
Found 689 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.


In [16]:
val_labels.shape

(689, 8)

In [17]:
X_train = trn.astype('float32')  # X_train.astype('float32')
X_test =  val.astype('float32')  # X_test.astype('float32')

# subtract mean and normalize
mean_image = np.mean(X_train, axis=0)
X_train -= mean_image
X_test -= mean_image
X_train /= 128.
X_test /= 128.

In [18]:
Y_train = trn_labels
Y_test  = val_labels

In [19]:
model = resnet.ResnetBuilder.build_resnet_50((img_channels, img_rows, img_cols), nb_classes)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
#model.optimizer.lr = 1e-4

In [20]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              verbose=1,
              validation_data=(X_test, Y_test),
              shuffle=True,
              callbacks=[lr_reducer, early_stopper, csv_logger]) # 
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        validation_data=(X_test, Y_test),
                        nb_epoch=nb_epoch, verbose=1, max_q_size=100,
                        callbacks=[lr_reducer, early_stopper, csv_logger])


Not using data augmentation.
Train on 3070 samples, validate on 689 samples
Epoch 1/200
3070/3070 [==============================] - 194s - loss: 5.6977 - acc: 0.6746 - val_loss: 8.9092 - val_acc: 0.1567
Epoch 2/200
3070/3070 [==============================] - 193s - loss: 3.8062 - acc: 0.8827 - val_loss: 4.5755 - val_acc: 0.5210
Epoch 3/200
3070/3070 [==============================] - 191s - loss: 2.7793 - acc: 0.9482 - val_loss: 5.1255 - val_acc: 0.5022
Epoch 4/200
3070/3070 [==============================] - 191s - loss: 2.2720 - acc: 0.9397 - val_loss: 4.1277 - val_acc: 0.4136
Epoch 5/200
3070/3070 [==============================] - 192s - loss: 1.8623 - acc: 0.9700 - val_loss: 3.6490 - val_acc: 0.5269
Epoch 6/200
3070/3070 [==============================] - 192s - loss: 1.5678 - acc: 0.9818 - val_loss: 3.2906 - val_acc: 0.5283
Epoch 7/200
3070/3070 [==============================] - 193s - loss: 1.3574 - acc: 0.9860 - val_loss: 3.0766 - val_acc: 0.6081
Epoch 8/200
3070/3070 [=====